# L=2 BMN SU2 matrix model

In [1]:
import numpy as np
import pylab
import matplotlib.pyplot as plt
import qiskit
from qiskit.quantum_info import Pauli
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit_algorithms import NumPyEigensolver
import time
import sys
sys.path.append('../utility')
from vqe_run import *
from qc_ansatze import *

# Load H

In [2]:
def open_n_process(file_link):
    file = open(file_link, "r")
    content = file.read()

    data = content.split(',\n')
    data[0] = data[0].split('\n')[1]
    data[-1] = data[-1].split('\n')[0]

    data_coeff = []
    data_str = []
    for i in range(len(data)):
        data_coeff.append(float(data[i].split('*')[0]))
        data_str.append(data[i].split('*')[1].split(' ')[1])
    return data_coeff, data_str
    
def check_min_max(data_coeff):
    print(f'Min absolute value is {np.round(np.min(np.abs(data_coeff[1:])),6)}')
    print(f'Max absolute value is {np.round(np.max(np.abs(data_coeff[1:])),5)}')
    print(f'Mean absolute value is {np.round(np.mean(np.abs(data_coeff[1:])),5)}')
    
def get_k_largest_ops(data_coeff, data_str, k):
    ind_list = np.argsort(np.abs(data_coeff[1:]))[-k:]
    ops = [data_str[1:][i] for i in ind_list]
    val= [np.round(data_coeff[1:][i],4) for i in ind_list]
    return ind_list, ops, val

In [3]:
gg = '05'
data_coeff, data_str = open_n_process(f"../utility/L2_BMN_l{gg}_Hpauli.txt")

In [4]:
Hpauli = list(zip(data_str, data_coeff))
H4q = SparsePauliOp.from_list(Hpauli)
# exactly diagonalize the system using numpy routines
solver = NumPyEigensolver(k=4)
exact_solution = solver.compute_eigenvalues(H4q)
#print("Exact Result of qubit hamiltonian:", np.real(exact_solution.eigenvalues))
E_exact = np.round(np.real(exact_solution.eigenvalues)[0],6)
print(f'{check_min_max(data_coeff)}')
print(f'E_exact = {E_exact}')

Min absolute value is 0.125
Max absolute value is 0.75
Mean absolute value is 0.35938
None
E_exact = 0.016901


In [5]:
Hpauli

[('IIIIIIIII', 5.625),
 ('ZIIIIIIII', -0.5),
 ('IZIIIIIII', -0.5),
 ('IIZIIIIII', -0.5),
 ('IIIZIIIII', -0.5),
 ('IIIIZIIII', -0.5),
 ('IIIIIZIII', -0.5),
 ('IIIIIIZII', -0.75),
 ('IIIIIIIZI', -0.75),
 ('IIIIIIIIZ', -0.75),
 ('XXIXXIIII', -0.12499999999999996),
 ('XIXXIXIII', -0.12499999999999996),
 ('IXXIXXIII', -0.12499999999999996),
 ('IIXIIIYXI', 0.24999999999999994),
 ('IIXIIIXYI', 0.24999999999999994),
 ('IIIIIXXXI', 0.24999999999999994),
 ('IIIIIXYYI', -0.24999999999999994),
 ('IXIIIIYZX', -0.24999999999999994),
 ('IXIIIIXZY', -0.24999999999999994),
 ('IIIIXIXZX', -0.24999999999999994),
 ('IIIIXIYZY', 0.24999999999999994),
 ('XIIIIIIYX', 0.24999999999999994),
 ('XIIIIIIXY', 0.24999999999999994),
 ('IIIXIIIXX', 0.24999999999999994),
 ('IIIXIIIYY', -0.24999999999999994)]

# EvolvedOperatorAnsatz

In [5]:
ind15, ops15, vals15 = get_k_largest_ops(data_coeff, data_str, 15)
ind20, ops20, vals20 = get_k_largest_ops(data_coeff, data_str, 20)

ops_Hp15 =[Pauli(f'{ops15[i]}') for i in range(len(ops15))]
ops_Hp20 =[Pauli(f'{ops20[i]}') for i in range(len(ops20))]
ops_H =[Pauli(f'{data_str[1:][i]}') for i in range(len(data_str[1:]))]

In [6]:
ev_op_Hp15 = EvolvedOperatorAnsatz(ops_Hp15, reps=1, insert_barriers=True)
ev_op_Hp20 = EvolvedOperatorAnsatz(ops_Hp20, reps=1, insert_barriers=True)
ev_op_H = EvolvedOperatorAnsatz(ops_H, reps=1, insert_barriers=True)

ev_op_Hp15_2f = EvolvedOperatorAnsatz(ops_Hp15, reps=2, insert_barriers=True)
ev_op_Hp20_2f = EvolvedOperatorAnsatz(ops_Hp20, reps=2, insert_barriers=True)
ev_op_H_2f = EvolvedOperatorAnsatz(ops_H, reps=2, insert_barriers=True)

ev_op_Hp15_3f = EvolvedOperatorAnsatz(ops_Hp15, reps=3, insert_barriers=True)
ev_op_Hp20_3f = EvolvedOperatorAnsatz(ops_Hp20, reps=3, insert_barriers=True)
ev_op_H_3f = EvolvedOperatorAnsatz(ops_H, reps=3, insert_barriers=True)

ev_op_Hp15_4f = EvolvedOperatorAnsatz(ops_Hp15, reps=4, insert_barriers=True)
ev_op_Hp20_4f = EvolvedOperatorAnsatz(ops_Hp20, reps=4, insert_barriers=True)
ev_op_H_4f = EvolvedOperatorAnsatz(ops_H, reps=4, insert_barriers=True)

ansatz_list = [ev_op_Hp15, ev_op_Hp20, ev_op_H, 
               ev_op_Hp15_2f, ev_op_Hp20_2f, ev_op_H_2f,
               ev_op_Hp15_3f, ev_op_Hp20_3f, ev_op_H_3f,
               ev_op_Hp15_4f, ev_op_Hp20_4f, ev_op_H_4f]

ansatz_names = ['ev_op_Hp15', 'ev_op_Hp20', 'ev_op_H', 
               'ev_op_Hp15_2f','ev_op_Hp20_2f','ev_op_H_2f',
               'ev_op_Hp15_3f','ev_op_Hp20_3f','ev_op_H_3f',
               'ev_op_Hp15_4f','ev_op_Hp20_4f','ev_op_H_4f']

print(f'number of params: {[ansatz_list[i].num_parameters for i in range(len(ansatz_list))]}')

number of params: [15, 20, 24, 30, 40, 48, 45, 60, 72, 60, 80, 96]


In [8]:
print(*list(zip(ops20, vals20)), sep = '\n')

('XIIIIIIYX', 0.25)
('IIIIXIYZY', 0.25)
('IIIIXIXZX', -0.25)
('IXIIIIXZY', -0.25)
('IXIIIIYZX', -0.25)
('IIIIIXYYI', -0.25)
('IIIIIXXXI', 0.25)
('IIXIIIXYI', 0.25)
('IIIXIIIYY', -0.25)
('IIIXIIIXX', 0.25)
('IIXIIIYXI', 0.25)
('IIIIIZIII', -0.5)
('IIIIZIIII', -0.5)
('IIIZIIIII', -0.5)
('IIZIIIIII', -0.5)
('IZIIIIIII', -0.5)
('ZIIIIIIII', -0.5)
('IIIIIIIIZ', -0.75)
('IIIIIIIZI', -0.75)
('IIIIIIZII', -0.75)


In [9]:
print(*list(zip(ops15, vals15)), sep = '\n')

('IIIIIXYYI', -0.25)
('IIIIIXXXI', 0.25)
('IIXIIIXYI', 0.25)
('IIIXIIIYY', -0.25)
('IIIXIIIXX', 0.25)
('IIXIIIYXI', 0.25)
('IIIIIZIII', -0.5)
('IIIIZIIII', -0.5)
('IIIZIIIII', -0.5)
('IIZIIIIII', -0.5)
('IZIIIIIII', -0.5)
('ZIIIIIIII', -0.5)
('IIIIIIIIZ', -0.75)
('IIIIIIIZI', -0.75)
('IIIIIIZII', -0.75)


# VQE

In [7]:
seed = 170
iterations = 2000
algorithm_globals.random_seed = seed

#estimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)
#storing values
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
def run_qve_w_specified_optimizer(iterations, optimizer, ansatz):
    opt = optimizer(maxiter = iterations)
    vqe = VQE(noiseless_estimator, ansatz, optimizer=opt, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H4q).eigenvalue.real
    print(f"VQE result: {result:.6f}")
    return result

# COBYLA

At step 3, with ev_op_Hp15_2f
VQE result: 0.194824
Length of this optimization 339, time taken = 35.927 

In [11]:
r_cobyla=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(iterations, COBYLA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_cobyla.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))
    

At step 0, with ev_op_Hp15
VQE result: 2.144043
Length of this optimization 181, time taken = 18.632 

At step 1, with ev_op_Hp20
VQE result: 0.240234
Length of this optimization 280, time taken = 42.091 

At step 2, with ev_op_H
VQE result: 2.361328
Length of this optimization 290, time taken = 59.611 

At step 3, with ev_op_Hp15_2f
VQE result: 0.194824
Length of this optimization 339, time taken = 51.793 

At step 4, with ev_op_Hp20_2f
VQE result: 0.221191
Length of this optimization 673, time taken = 133.555 

At step 5, with ev_op_H_2f
VQE result: 0.397461
Length of this optimization 775, time taken = 181.407 

At step 6, with ev_op_Hp15_3f
VQE result: 0.206055
Length of this optimization 612, time taken = 97.434 

At step 7, with ev_op_Hp20_3f
VQE result: 1.229004
Length of this optimization 864, time taken = 234.766 

At step 8, with ev_op_H_3f
VQE result: 2.872559
Length of this optimization 1004, time taken = 337.944 

At step 9, with ev_op_Hp15_4f
VQE result: 0.248535
Length o

In [12]:
df1 = pd.concat([r_cobyla[i] for i in range(len(r_cobyla))], axis = 1)
df1.to_csv(f'L2_l{gg}_op_ev_cobyla.csv')

# SPSA

At step 4, with ev_op_Hp20_2f
VQE result: 0.188477
Length of this optimization 3551, time taken = 525.911 

In [13]:
r_spsa=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(iterations, SPSA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_spsa.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15
VQE result: 0.222168
Length of this optimization 4051, time taken = 264.718 

At step 1, with ev_op_Hp20
VQE result: 0.056641
Length of this optimization 4051, time taken = 415.831 

At step 2, with ev_op_H
VQE result: 0.331055
Length of this optimization 4051, time taken = 465.284 

At step 3, with ev_op_Hp15_2f
VQE result: 0.237305
Length of this optimization 4051, time taken = 363.041 

At step 4, with ev_op_Hp20_2f
VQE result: 0.187988
Length of this optimization 4051, time taken = 601.143 

At step 5, with ev_op_H_2f
VQE result: 2.151855
Length of this optimization 4051, time taken = 709.495 

At step 6, with ev_op_Hp15_3f
VQE result: 0.263184
Length of this optimization 4051, time taken = 467.699 

At step 7, with ev_op_Hp20_3f
VQE result: 0.320312
Length of this optimization 4051, time taken = 779.424 

At step 8, with ev_op_H_3f
VQE result: 1.846680
Length of this optimization 4051, time taken = 923.667 

At step 9, with ev_op_Hp15_4f
VQE result: 0.26

In [14]:
df2 = pd.concat([r_spsa[i] for i in range(len(r_spsa))], axis = 1)
df2.to_csv(f'L2_l{gg}_op_ev_spsa.csv')

## Retrain best ansatz

At step 1, with ev_op_Hp20
VQE result: 0.056641
Length of this optimization 4051, time taken = 415.831 

In [9]:
#Increasing the number of iters makes it worse
iterations = 2500
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_a = counts
values_a = values

VQE result: 2.001465
Length of this optimization 5051, time taken = 648.404 



In [10]:
#Reducing the number of iters makes it better
iterations = 1500
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_a = counts
values_a = values

VQE result: 0.042969
Length of this optimization 3051, time taken = 420.838 



In [11]:
#Reducing the number of iters makes slightly worse compared to before
iterations = 1200
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_b = counts
values_b = values

VQE result: 0.047363
Length of this optimization 2451, time taken = 321.283 



In [12]:
#Reducing the number of iters makes it better
iterations = 1750
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_c = counts
values_c = values

VQE result: 0.035645
Length of this optimization 3551, time taken = 491.148 



In [14]:
df = pd.DataFrame({'ev_op_Hp20': values_c})
df.to_csv(f'results/L2_l{gg}_op_ev_spsa_best.csv')

# SLSQP

In [15]:
r_slsqp=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(iterations, SLSQP, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_slsqp.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15
VQE result: 5.918457
Length of this optimization 16, time taken = 2.145 

At step 1, with ev_op_Hp20
VQE result: 5.286133
Length of this optimization 21, time taken = 3.563 

At step 2, with ev_op_H
VQE result: 6.044434
Length of this optimization 25, time taken = 4.138 

At step 3, with ev_op_Hp15_2f
VQE result: 4.714355
Length of this optimization 31, time taken = 4.051 

At step 4, with ev_op_Hp20_2f
VQE result: 5.350098
Length of this optimization 41, time taken = 8.891 

At step 5, with ev_op_H_2f
VQE result: 5.500488
Length of this optimization 49, time taken = 12.773 

At step 6, with ev_op_Hp15_3f
VQE result: 3.947754
Length of this optimization 46, time taken = 7.832 

At step 7, with ev_op_Hp20_3f
VQE result: 5.881348
Length of this optimization 61, time taken = 17.918 

At step 8, with ev_op_H_3f
VQE result: 5.444336
Length of this optimization 73, time taken = 28.208 

At step 9, with ev_op_Hp15_4f
VQE result: 4.443848
Length of this optimization 

In [16]:
df3 = pd.concat([r_slsqp[i] for i in range(len(r_slsqp))], axis = 1)
df3.to_csv(f'L2_l{gg}_op_ev_slsqp.csv')